<a href="https://colab.research.google.com/github/Dicere/Machine_Learning_algorithms_from_scratch/blob/main/Linear_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [338]:
import numpy as np
import pandas as pd

In [339]:
from sklearn.datasets import load_diabetes

data = load_diabetes(as_frame=True)
X, y = data['data'], data['target']

In [469]:
class MyLineReg():
    def __init__(self, n_iter=100, learning_rate=0.1,weights=None,verbose=1):         
        self.n_iter= n_iter
        self.learning_rate = learning_rate
        self.weights = weights
        self.verbose = verbose
        
    def __str__(self):
        attributes = ', '.join(f"{key}={value}" for key, value in vars(self).items())
        return f"MyLineReg class: {attributes}"

    def __repr__(self):
        attributes = ', '.join(f"{key}={value}" for key, value in vars(self).items())
        return f"MyLineReg class: {attributes}"

    def fit(self,X_fit:pd.DataFrame,y_fit:pd.Series,n_iter=None,learning_rate=None,verbose=None):
      X = X_fit.copy()
      y = y_fit.copy()
      X.insert(0, "One", 1)
      n = X.shape[0]
      self.weights =  np.ones(X.shape[1])

      if n_iter is not None:
        self.n_iter = n_iter
      if learning_rate is not None:
        self.learning_rate = learning_rate
      if verbose is not None:
        self.verbose = verbose

      for i in range(self.n_iter):
        
        y_pred_vec = X.to_numpy()*self.weights
        y_pred = np.sum(y_pred_vec,axis=1)
        error = (y_pred - y).to_numpy().reshape(n,1)
        grad_vec = (2/n)*(error*(X.to_numpy()))
        grad = np.sum(grad_vec,axis=0)

        if i == 0 and self.verbose != 0:
          print(f"start:{((y_pred - y)**2).sum()}")

        self.weights = self.weights - (self.learning_rate*grad)

        if i !=0 and self.verbose != 0:
          if i % self.verbose == 0:
            print(f"{i}:{((y_pred - y)**2).sum()}")

    def get_coef(self):
        return self.weights[1:]

    def predict(self,X_test:pd.DataFrame):
        X = X_test.copy()
        X.insert(0, "One", 1)
        y_pred_vec = X.to_numpy()*self.weights
        return np.sum(y_pred_vec,axis=1)

In [470]:
instance = MyLineReg()

In [472]:
instance.fit(X,y,n_iter=100,learning_rate=0.1,verbose=0)

In [473]:
instance.predict(X.iloc[:3])

array([156.33176834, 140.46994197, 154.19895694])